# 새로시작

In [82]:

import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [108]:
#불러오기
gyeonggi_cafe = pd.read_csv('gyeonggi_cafe.csv')

In [84]:
gyeonggi_cafe = gyeonggi_cafe.iloc[:,[1,2]]
gyeonggi_cafe.dropna(inplace=True)
negative_df = gyeonggi_cafe[gyeonggi_cafe['별점'] <= 2]
positive_df = gyeonggi_cafe[gyeonggi_cafe['별점'] >= 4]
negative_df = negative_df.drop('별점',axis=1)
positive_df = positive_df.drop('별점',axis=1)

In [85]:
from hanspell import spell_checker

In [111]:
gyeonggi_cafe[gyeonggi_cafe['별점'] <= 2]

In [125]:
df1[df1['리뷰'] =='감사합니다. 다시 올게요.']

,Unnamed: 0,리뷰,별점,리뷰자 평균 평점,리뷰자 누적 리뷰 개수,영업 시간,이미지url,가게명,가게 주소,메뉴,카카오 url
11546,21,감사합니다. 다시 올게요.,2,4.6,96,매일 09:00 ~ 22:00,'//t1.kakaocdn.net/thumb/T800x0.q80/?fname=htt...,빵에갸또,경기 남양주시 두물로39번길 25 1층 (우)12105,아메리카노(HOT) 라떼(HOT) 카푸치노(HOT) 얼그레이 프렌치 블루 바닐라 밀...,https://place.map.kakao.com/714041699


In [105]:
for i in negative_df['리뷰']:
    print(i)
negative_df.iloc[1339]

아메리카노 하나에 8천원강 앞이라 지금 같은 날씨에 가면 날벌레 떼가 우글우글사람은 복작복작인테리어는 조잡
뷰 맛집케익 정말 맛없음
곡 선곡이 너무 별루네요.  곡 분위기 바꾸면 훨씬 고급질듯…
커피는 맛없고 빵은 그냥그럼 인테리어 좋고 뷰가 좋음불친절함
불친절
형광노랑 티셔츠에 검은 벙거지모자 쓰신 장발의 남자 주차요원분 계속 근무하시면 다시는 안갈것같네요. 카페 앞의 도로가 좁아서 주차자리로 일단 전면으로 들어갔다가 다시 후면 주차를 하려는데 저희 아버지한테 다른 차들 주차된거 보라고 후면으로 주차하라고 하대하면서 얘기하네요. 주변에 더 깔끔하면서 같은 뷰의 카페들 많아요. 거기로 가세요
여기원래 나인블럭이엇던 곳인가요?!?
커피가 너무너무너무 맛없어요....ㅜㅜㅜ 그리고 주말엔 자리전쟁 주차전쟁...실내는 창이 너무 더러워서 뷰가 좋다고 하기도 어려워요
납품 케이크인데 9천원..? 뷰도 별로였는데 음료랑 디저트는 왜이렇게 비싼지.. 뷰 보기 전에 창문에 거미줄부터 어떻게 해야 할 듯ㅠ
넘 바빠서 메뉴 주문 틀리시구… 커피는 그냥 그렇습니다… 디저트는 대량으로 있어서 썩 맛있진 않아뇸… 뷰 때문에 오시는 거면 다 감안하시구 오셔용
뷰맛집, 커피먹으러 오기 좋을거같음.그냥 배고픈 배를 움켜쥐고 가 파스타와 브런치시킨 내 잘못. 매우 기대하고 갔으나...만오천원이라는 가격치고 파스타도... 브런치도... 8000원짜리같음.. 면은 따로놀고, 브런치도 원물그대로쓰는데도 해쉬브라운이 맛없기는 처음ㅎ.과일 제일 맛있었다...어려워도 조리하는 분 채용하시던지 디저트랑 커피만 파시는게 가게에 이익일듯..
스윗스트로베리  너무 맹맹해요
아메리카노 8천원 너무비쌈
음료진짜 별로
뷰빼면 시체인데음료가 심각히 맛없음
주차장은넓어서좋은데주말이라 복잡함ᆢ아이스아멜카는 신맛나는커피네요 ㆍ비싸요 ㆍ
북한강 보이는 카페 뷰가 다 똑같지~~밖에 주차된 차가 너~~~무 많아서 호기심에 들어갔는데  아메리카노 8천원은 좀 그렇다
뷰가 좋고 커피는,,, 가성비 별로입니다 ㅎ근데 여기 방역수칙

리뷰    리모델링하고 좌석도 더 불편해지고.. 케익 맛도 떨어지고..
Name: 5326, dtype: object

In [86]:
negative_df['리뷰'].shape
negative_list = negative_df['리뷰']
negative_list= list(negative_list)
negative_reviews= [review.split('.') for review in negative_list]
negative_reviews = sum(negative_reviews, [])
negative_list = []
for i in negative_reviews:
    if len(i) > 0 and len(i)< 30 : 
        negative_list.append(i.strip())
    else:
        pass

In [87]:
tokenizer1 = Tokenizer()
tokenizer1.fit_on_texts(negative_list)
vocab_size = len(tokenizer1.word_index) + 1
print('단어 집합의 크기 : %d' % vocab_size)

단어 집합의 크기 : 13116


In [97]:
sequences = list()
for line in negative_list: # 줄바꿈 문자를 기준으로 문장 토큰화
    encoded = tokenizer1.texts_to_sequences([line])[0]
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

print('학습에 사용할 샘플의 개수: %d' % len(sequences))

학습에 사용할 샘플의 개수: 20809


In [88]:
import tensorflow as tf
model = tf.keras.models.load_model('negative_model.h5', compile=False)


In [89]:
def sentence_generation(model, tokenizer, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word
    sentence = ''

    # n번 반복
    for _ in range(n):
        encoded = tokenizer1.texts_to_sequences([current_word])[0]
        encoded = pad_sequences([encoded], maxlen=max_len-1, padding='pre')

        # 입력한 X(현재 단어)에 대해서 y를 예측하고 y(예측한 단어)를 result에 저장.
        result = model.predict(encoded, verbose=0)
        result = np.argmax(result, axis=1)

        for word, index in tokenizer1.word_index.items(): 
            # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
            if index == result:
                break

        # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        current_word = current_word + ' '  + word

        # 예측 단어를 문장에 저장
        sentence = sentence + ' ' + word

    sentence = init_word + sentence
    return sentence

In [127]:
print(sentence_generation(model, tokenizer1, '주차장',5))

주차장 있다 없어요 8000원인데 매우 한입먹고
